In [2]:
import pandas as pd

merged_merchant_with_categories = pd.read_parquet('../data/curated/merged_merchant_info_with_categories.parquet')

In [3]:
merged_merchant_with_categories.head()

,merchant_abn,category_label,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions
0,10023283211,Furniture and Home Furnishing,0.18,0.0,216.736529,155.285714
21,10142254217,"Media and Technology, Services",4.22,0.0,37.716001,144.571429
42,10165489824,Retail Goods,4.40,0.0,11236.094771,0.454545
53,10187291046,"Automotives, Retail Goods, Services",3.29,0.0,119.526004,16.800000
73,10192359162,Retail Goods,6.33,0.0,462.329450,19.250000


In [4]:
consumer_tbl = pd.read_csv('../data/raw/tbl_consumer.csv', delimiter='|')
userdf = pd.read_parquet('../data/raw/consumer_user_details.parquet')
merged = consumer_tbl.merge(userdf, on='consumer_id', how='left')

In [5]:
test_df = pd.read_parquet('../test/')

In [6]:
merged_with_test = merged.merge(test_df, on='user_id', how='left')
merged_with_test_dropped = merged_with_test.dropna()

In [7]:
# Dropped na since no one has bought from these merchants
merged_with_abn = merged_with_test_dropped.merge(merged_merchant_with_categories, on='merchant_abn', how='left')
merged_with_abn_dropped = merged_with_abn.dropna()

In [8]:
columns_to_drop = ['name', 'address', 'state', 'gender', 'dollar_value', 'order_id', 'order_datetime', 'category_label']

cleaned_merged_with_abn = merged_with_abn_dropped.drop(columns=columns_to_drop)

In [9]:
census_data = pd.read_csv('../data/curated/2021Census_G02_AUST_POA_curated.csv')
census_data = census_data.rename(columns={'POA_CODE_2021': 'postcode'})
census_data.head()
merged_with_census = cleaned_merged_with_abn.merge(census_data, on='postcode', how='left')

print(merged_with_census.head())
merged_with_census.isnull().sum()

   postcode  consumer_id  user_id  merchant_abn  take_rate_value  \
0      6935      1195503        1  2.800049e+10             4.24   
1      6935      1195503        1  8.369064e+10             3.15   
2      6935      1195503        1  9.668077e+10             5.91   
3      6935      1195503        1  4.989171e+10             5.80   
4      6935      1195503        1  6.240384e+10             6.46   

   fraud_probability  average_monthly_revenue  average_monthly_transactions  \
0           0.000000               245.258150                    180.523810   
1           0.000000                88.512653                   1707.238095   
2          29.555245               316.660874                   1482.571429   
3           0.000000                28.889602                  11786.952381   
4           0.000000                34.467794                    184.571429   

   Median_age_persons  Median_mortgage_repay_monthly  Median_rent_weekly  \
0                 NaN                   

postcode                               0
consumer_id                            0
user_id                                0
merchant_abn                           0
take_rate_value                        0
fraud_probability                      0
average_monthly_revenue                0
average_monthly_transactions           0
Median_age_persons               2244225
Median_mortgage_repay_monthly    2244225
Median_rent_weekly               2244225
Median_tot_hhd_inc_weekly        2244225
dtype: int64

In [10]:
merged_with_census_dropped = merged_with_census.dropna()
merged_with_census_dropped.head()

,postcode,consumer_id,user_id,merchant_abn,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly
553,2782,179208,2,8.077982e+10,4.71,0.0,34.567393,1735.142857,50.0,2080.0,430.0,1528.0
554,2782,179208,2,2.864764e+10,3.32,0.0,263.829390,23.476190,50.0,2080.0,430.0,1528.0
555,2782,179208,2,3.581349e+10,5.66,0.0,45.885892,182.761905,50.0,2080.0,430.0,1528.0
556,2782,179208,2,3.809009e+10,3.24,0.0,172.158074,2467.428571,50.0,2080.0,430.0,1528.0
557,2782,179208,2,3.897149e+10,3.61,0.0,166.642856,269.761905,50.0,2080.0,430.0,1528.0


In [11]:
aggregated_df = merged_with_census_dropped.groupby('merchant_abn').mean()
aggregated_df = aggregated_df.reset_index()
aggregated_df = aggregated_df.drop(columns=['consumer_id', 'postcode', 'user_id'])

aggregated_df.head()

,merchant_abn,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly
0,1.002328e+10,0.18,0.0,216.736529,155.285714,43.367701,1544.717295,297.540281,1598.090540
1,1.014225e+10,4.22,0.0,37.716001,144.571429,43.285770,1566.667060,299.012972,1609.036557
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,39.500000,1735.250000,332.500000,1855.250000
3,1.018729e+10,3.29,0.0,119.526004,16.800000,43.839041,1563.356164,294.119863,1595.914384
4,1.019236e+10,6.33,0.0,462.329450,19.250000,43.287926,1595.092879,309.244582,1657.421053


In [12]:
merchant_categories = pd.read_parquet('../data/curated/merchant_categories.parquet')
merchant_categories = merchant_categories.drop(columns=['category', 'revenue_rate', 'fraud_probability', 'take_rate_value'])
category_df = aggregated_df.merge(merchant_categories, on='merchant_abn', how='left')



In [22]:
merchant_fraud = pd.read_csv('../data/raw/merchant_fraud_probability.csv')
final_df = category_df.merge(merchant_fraud, on='merchant_abn', how='left')
final_df['fraud_probability_y'] = final_df['fraud_probability_y'].apply(lambda x: 0 if pd.isna(x) else 1)
final_df = final_df.drop(columns = ['order_datetime'])
final_df = final_df.rename(columns={'fraud_probability_x': 'consumer_fraud', 'fraud_probability_y': 'merchant_fraud'})


In [23]:
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,1.002328e+10,0.18,0.0,216.736529,155.285714,43.367701,1544.717295,297.540281,1598.090540,Felis Limited,Furniture and Home Furnishing,0
1,1.014225e+10,4.22,0.0,37.716001,144.571429,43.285770,1566.667060,299.012972,1609.036557,Arcu Ac Orci Corporation,"Media and Technology, Services",0
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,39.500000,1735.250000,332.500000,1855.250000,Nunc Sed Company,Retail Goods,0
3,1.018729e+10,3.29,0.0,119.526004,16.800000,43.839041,1563.356164,294.119863,1595.914384,Ultricies Dignissim Lacus Foundation,"Automotives, Retail Goods, Services",0
4,1.019236e+10,6.33,0.0,462.329450,19.250000,43.287926,1595.092879,309.244582,1657.421053,Enim Condimentum PC,Retail Goods,0


In [24]:
final_df = final_df.drop(columns=['name'])
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,category_label,merchant_fraud
0,1.002328e+10,0.18,0.0,216.736529,155.285714,43.367701,1544.717295,297.540281,1598.090540,Furniture and Home Furnishing,0
1,1.014225e+10,4.22,0.0,37.716001,144.571429,43.285770,1566.667060,299.012972,1609.036557,"Media and Technology, Services",0
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,39.500000,1735.250000,332.500000,1855.250000,Retail Goods,0
3,1.018729e+10,3.29,0.0,119.526004,16.800000,43.839041,1563.356164,294.119863,1595.914384,"Automotives, Retail Goods, Services",0
4,1.019236e+10,6.33,0.0,462.329450,19.250000,43.287926,1595.092879,309.244582,1657.421053,Retail Goods,0


In [25]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Assuming final_df is your DataFrame

# Columns to exclude from standardization
exclude_columns = ['merchant_abn', 'consumer_fraud', 'merchant_fraud', 'category_label']

# Columns to standardize
columns_to_standardize = [col for col in final_df.columns if col not in exclude_columns]

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Apply the scaler to the columns to be standardized
final_df[columns_to_standardize] = scaler.fit_transform(final_df[columns_to_standardize])

# Display the first few rows of the updated DataFrame to verify
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,category_label,merchant_fraud
0,1.002328e+10,0.011594,0.0,0.004023,0.011255,0.467649,0.154487,0.566743,0.341571,Furniture and Home Furnishing,0
1,1.014225e+10,0.597101,0.0,0.000571,0.010478,0.465308,0.156682,0.569549,0.346294,"Media and Technology, Services",0
2,1.016549e+10,0.623188,0.0,0.216470,0.000024,0.357143,0.173542,0.633333,0.452535,Retail Goods,0
3,1.018729e+10,0.462319,0.0,0.002148,0.001210,0.481115,0.156351,0.560228,0.340632,"Automotives, Retail Goods, Services",0
4,1.019236e+10,0.902899,0.0,0.008758,0.001387,0.465369,0.159525,0.589037,0.367172,Retail Goods,0


In [26]:
import pandas as pd

# Assuming final_df is your DataFrame

# Define the weights for each column
weights = {
    'merchant_fraud': -1,
    'average_monthly_revenue': 0.9,
    'take_rate_value': 0.7,
    'consumer_fraud': -0.6,
    'average_monthly_transactions': 0.5,
    'Median_tot_hhd_inc_weekly': 0.2,
    'Median_age_persons': 0.15,
    'Median_mortgage_repay_monthly': 0.1,
    'Median_rent_weekly': 0.1
}

# Calculate the score for each row
final_df['score'] = (
    final_df['merchant_fraud'] * weights['merchant_fraud'] +
    final_df['average_monthly_revenue'] * weights['average_monthly_revenue'] +
    final_df['take_rate_value'] * weights['take_rate_value'] +
    final_df['consumer_fraud'] * weights['consumer_fraud'] +
    final_df['average_monthly_transactions'] * weights['average_monthly_transactions'] +
    final_df['Median_tot_hhd_inc_weekly'] * weights['Median_tot_hhd_inc_weekly'] +
    final_df['Median_age_persons'] * weights['Median_age_persons'] +
    final_df['Median_mortgage_repay_monthly'] * weights['Median_mortgage_repay_monthly'] +
    final_df['Median_rent_weekly'] * weights['Median_rent_weekly']
)

# Display the first few rows of the updated DataFrame to verify
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,category_label,merchant_fraud,score
0,1.002328e+10,0.011594,0.0,0.004023,0.011255,0.467649,0.154487,0.566743,0.341571,Furniture and Home Furnishing,0,0.227948
1,1.014225e+10,0.597101,0.0,0.000571,0.010478,0.465308,0.156682,0.569549,0.346294,"Media and Technology, Services",0,0.635402
2,1.016549e+10,0.623188,0.0,0.216470,0.000024,0.357143,0.173542,0.633333,0.452535,Retail Goods,0,0.855833
3,1.018729e+10,0.462319,0.0,0.002148,0.001210,0.481115,0.156351,0.560228,0.340632,"Automotives, Retail Goods, Services",0,0.538113
4,1.019236e+10,0.902899,0.0,0.008758,0.001387,0.465369,0.159525,0.589037,0.367172,Retail Goods,0,0.858700


In [30]:
import pandas as pd

# Assuming final_df is your DataFrame and the 'score' column has already been calculated

# Sort the DataFrame by the 'score' column in descending order
sorted_df = final_df.sort_values(by='score', ascending=False)

# Select the top 100 merchants
top_100_merchants = sorted_df.head(100)

# Display the top 100 merchants
top_100_merchants.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,category_label,merchant_fraud,score
1580,3.915015e+10,0.998551,0.0,0.734199,0.000063,0.257143,0.140814,0.476190,0.416613,"Automotives, Services",0,1.543391
3728,8.657848e+10,0.917391,0.0,0.000520,0.941836,0.469367,0.156114,0.572560,0.345250,"Automotives, Retail Goods, Services",0,1.325882
1870,4.562922e+10,0.997101,0.0,0.000549,0.788408,0.468463,0.155675,0.571345,0.344656,"Automotives, Retail Goods",0,1.304572
971,2.906845e+10,0.460870,0.0,0.712237,0.000063,1.000000,0.000000,0.000000,0.838188,"Automotives, Services",0,1.281291
16,1.040454e+10,0.989855,0.0,0.438915,0.000063,0.428571,0.077308,0.323810,0.261273,"Automotives, Services",0,1.244606


In [51]:
import pandas as pd

# Assuming final_df is your DataFrame and the 'score' column has already been calculated

# Define the category labels
categories = ['Retail Goods', 'Media and Technology', 'Furniture and Home Furnishing', 'Services', 'Automotives']

# Initialize variables to store the top 10 merchants for each category
top_10_retail_goods = None
top_10_media_technology = None
top_10_furniture_home_furnishing = None
top_10_services = None
top_10_automotives = None

# Loop through each category and get the top 10 merchants
for category in categories:
    # Filter and sort the DataFrame by the current category and score
    top_10_merchants = final_df[final_df['category_label'] == category].sort_values(by='score', ascending=False).head(10)
    
    # Store the result in the corresponding variable
    if category == 'Retail Goods':
        top_10_retail_goods = top_10_merchants
    elif category == 'Media and Technology':
        top_10_media_technology = top_10_merchants
    elif category == 'Furniture and Home Furnishing':
        top_10_furniture_home_furnishing = top_10_merchants
    elif category == 'Services':
        top_10_services = top_10_merchants
    elif category == 'Automotives':
        top_10_automotives = top_10_merchants

# Display the top 10 merchants for each category
print("Top 10 merchants in 'Retail Goods':")
print(top_10_retail_goods)
print("\n")

print("Top 10 merchants in 'Media and Technology':")
print(top_10_media_technology)
print("\n")

print("Top 10 merchants in 'Furniture and Home Furnishing':")
print(top_10_furniture_home_furnishing)
print("\n")

print("Top 10 merchants in 'Services':")
print(top_10_services)
print("\n")

print("Top 10 merchants in 'Automotives':")
print(top_10_automotives)
print("\n")

Top 10 merchants in 'Retail Goods':
      merchant_abn  take_rate_value  consumer_fraud  average_monthly_revenue  \
2679  6.414355e+10         0.837681             0.0                 0.400320   
4288  9.759036e+10         0.926087             0.0                 0.301079   
25    1.059630e+10         0.976812             0.0                 0.236222   
36    1.088104e+10         0.889855             0.0                 0.225276   
1810  4.434579e+10         0.865217             0.0                 0.280284   
2690  6.440360e+10         0.900000             0.0                 0.001350   
2815  6.721352e+10         0.647826             0.0                 0.309200   
3378  7.930698e+10         0.981159             0.0                 0.038628   
2865  6.823387e+10         0.808696             0.0                 0.170962   
3737  8.674450e+10         0.947826             0.0                 0.057935   

      average_monthly_transactions  Median_age_persons  \
2679                     